## ETAPA 1: CARGA INICIAL DE DATOS

Cargar los datasets desde Cloud Storage

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL_SUTRAN").getOrCreate()

# Ruta base en Cloud Storage
bucket_path = "gs://sutran-bucket-2025/raw/"

# Leer los 3 datasets
df_personas = spark.read.option("header", True).option("inferSchema", True).csv(bucket_path + "BBDD_ONSV-PERSONAS_2021-2023.csv")
df_vehiculos = spark.read.option("header", True).option("inferSchema", True).csv(bucket_path + "BBDD_ONSV-VEHICULOS_2021-2023.csv")
df_siniestros = spark.read.option("header", True).option("inferSchema", True).csv(bucket_path + "BBDD_ONSV-SINIESTROS_2021-2023.csv")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/17 16:04:46 INFO SparkEnv: Registering MapOutputTracker
25/11/17 16:04:46 INFO SparkEnv: Registering BlockManagerMaster
25/11/17 16:04:47 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
25/11/17 16:04:47 INFO SparkEnv: Registering OutputCommitCoordinator


Verificamos que se cargaron bien

In [ ]:
df_personas.printSchema()
df_vehiculos.printSchema()
df_siniestros.printSchema()

root
 |-- CODIGO_SINIESTRO: string (nullable = true)
 |-- CODIGO_VEHICULO: string (nullable = true)
 |-- CODIGO_PERSONA: string (nullable = true)
 |-- DEPARTAMENTO: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- DISTRITO: string (nullable = true)
 |-- TIPO_PERSONA: string (nullable = true)
 |-- GRAVEDAD: string (nullable = true)
 |-- LUGAR_ATENCION_LESIONADO: string (nullable = true)
 |-- LUGAR_DE_DEFUNCION: string (nullable = true)
 |-- SITUACION_DE_PERSONA: string (nullable = true)
 |-- PAIS_DE_NACIONALIDAD: string (nullable = true)
 |-- EDAD: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- POSEE_LICENCIA: string (nullable = true)
 |-- ESTADO_LICENCIA: string (nullable = true)
 |-- CLASE_LICENCIA: string (nullable = true)
 |-- SE_SOMETIO_A_DOSAJE_ETILICO_CUALITATIVO: string (nullable = true)
 |-- RESULTADO_DEL_DOSAJE_ETILICO_CUALITATIVO: string (nullable = true)
 |-- SE_SOMETIO_A_DOSAJE_ETILICO_CUANTITATIVO: string (nullable = true)
 |-- VEHI

In [ ]:
df_personas.show(5)
df_vehiculos.show(5)
df_siniestros.show(5)

25/11/17 16:05:27 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------------+---------------+-----------------+------------+---------+----------+----------------+---------+------------------------+------------------+--------------------+--------------------+----+---------+--------------+---------------+--------------+---------------------------------------+----------------------------------------+----------------------------------------+-----------------+----------+----+-----+-------+----+------------------+--------------------+--------------------+--------------------+-------------------+-------------+
|CODIGO_SINIESTRO|CODIGO_VEHICULO|   CODIGO_PERSONA|DEPARTAMENTO|PROVINCIA|  DISTRITO|    TIPO_PERSONA| GRAVEDAD|LUGAR_ATENCION_LESIONADO|LUGAR_DE_DEFUNCION|SITUACION_DE_PERSONA|PAIS_DE_NACIONALIDAD|EDAD|     SEXO|POSEE_LICENCIA|ESTADO_LICENCIA|CLASE_LICENCIA|SE_SOMETIO_A_DOSAJE_ETILICO_CUALITATIVO|RESULTADO_DEL_DOSAJE_ETILICO_CUALITATIVO|SE_SOMETIO_A_DOSAJE_ETILICO_CUANTITATIVO|         VEHICULO|     FECHA|ANIO|  MES|    DIA|HORA|CLASE_DE_SINI

## ETAPA 2 – TRANSFORMACIÓN EN MODELO ESTRELLA

### 1. Iniciar sesión Spark

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL_SUTRAN").getOrCreate()


### 2. Leer y limpiar los CSV desde Cloud Storage

2.1 Definir función para limpiar columnas (quita BOM, espacios, etc.)

In [ ]:
from pyspark.sql.functions import col

def limpiar_columnas(df):
    return df.select([col(c).alias(c.strip().replace("ï»¿", "").replace("\ufeff", "")) for c in df.columns])

2.2 Leer datasets

In [ ]:
bucket = "sutran-bucket-2025"
path_raw = f"gs://{bucket}/raw/"

# Personas
personas = spark.read.option("header", "true") \
    .option("encoding", "ISO-8859-1") \
    .csv(path_raw + "BBDD_ONSV-PERSONAS_2021-2023.csv")

personas_clean = limpiar_columnas(personas)

# Vehículos
vehiculos = spark.read.option("header", "true") \
    .option("encoding", "ISO-8859-1") \
    .csv(path_raw + "BBDD_ONSV-VEHICULOS_2021-2023.csv")

vehiculos_clean = limpiar_columnas(vehiculos)

# Siniestros
siniestros = spark.read.option("header", "true") \
    .option("encoding", "ISO-8859-1") \
    .csv(path_raw + "BBDD_ONSV-SINIESTROS_2021-2023.csv")

siniestros_clean = limpiar_columnas(siniestros)

# Cast columnas numéricas en siniestros
siniestros_clean = siniestros_clean \
    .withColumn("CANTIDAD_DE_FALLECIDOS", col("CANTIDAD_DE_FALLECIDOS").cast("int")) \
    .withColumn("CANTIDAD_DE_LESIONADOS", col("CANTIDAD_DE_LESIONADOS").cast("int")) \
    .withColumn("CANTIDAD_DE_VEHICULOS_DANADOS", col("CANTIDAD_DE_VEHICULOS_DANADOS").cast("int"))

### 3. Crear tablas del modelo estrella

3.1 dim_persona

In [ ]:
dim_persona = personas_clean.select(
    "CODIGO_PERSONA",
    "SEXO",
    "EDAD",
    "CLASE_LICENCIA",
    "PAIS_DE_NACIONALIDAD",
    "POSEE_LICENCIA",
    "RESULTADO_DEL_DOSAJE_ETILICO_CUALITATIVO", #Aca se hizo cambio registrar en bitácora
    "SITUACION_DE_PERSONA",
    "GRAVEDAD"
).dropDuplicates(["CODIGO_PERSONA"])


3.2 dim_vehiculo

In [ ]:
dim_vehiculo = vehiculos_clean.select(
    "CODIGO_VEHICULO",
    "VEHICULO",
    "MODALIDAD_DE_TRANSPORTE",
    "TIPO_SEGURO",
    "COMPANIA_SEGURO",
    "ESTADO_CITV",
    "ESTADO_SOAT",
    "ESTADO_MODALIDAD",
    "SITUACION_VEHICULO"
).dropDuplicates(["CODIGO_VEHICULO"])


3.3 dim_tiempo

In [ ]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

siniestros_clean = siniestros_clean.withColumn("FECHA", to_date("FECHA_SINIESTRO", "yyyy-MM-dd"))

dim_tiempo = siniestros_clean.select("FECHA").distinct() \
    .withColumn("ANO", year("FECHA")) \
    .withColumn("MES", month("FECHA")) \
    .withColumn("DIA", dayofmonth("FECHA")) \
    .withColumn("ID_TIEMPO", col("FECHA").cast("string"))


3.4 dim_tipo_via

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

dim_tipo_via = siniestros_clean.select(
    "TIPO_DE_VIA",
    "PERFIL_LONGITUDINAL_VIA",
    "SUPERFICIE_DE_CALZADA",
    "ZONIFICACION"
).dropDuplicates()

dim_tipo_via = dim_tipo_via.withColumn("ID_TIPO_VIA", monotonically_increasing_id())

3.5 f_siniestro (tabla de hechos)

In [ ]:
f_siniestro = siniestros_clean.select(
    "CODIGO_SINIESTRO",
    "FECHA",
    "TIPO_DE_VIA",
    "CAUSA_FACTOR_PRINCIPAL",
    "CLASE_SINIESTRO",
    "COORDENADAS_LATITUD",
    "CANTIDAD_DE_FALLECIDOS",
    "CANTIDAD_DE_LESIONADOS",
    "CANTIDAD_DE_VEHICULOS_DANADOS"
)

### 4. Cargar tablas a BigQuery

In [ ]:
# Proyecto y dataset
bq_project = "shaped-icon-478404-p0"
bq_dataset = "bi_sutran"


4.1 Subir dim_persona

In [25]:
dim_persona.write \
    .format("bigquery") \
    .option("table", f"{bq_project}.{bq_dataset}.dim_persona") \
    .option("temporaryGcsBucket", "sutran-bucket-2025") \
    .mode("overwrite") \
    .save()


4.2 Lo mismo para las demás dimensiones

In [26]:
dim_vehiculo.write.format("bigquery").option("table", f"{bq_project}.{bq_dataset}.dim_vehiculo").option("temporaryGcsBucket", "sutran-bucket-2025").mode("overwrite").save()

dim_tiempo.write.format("bigquery").option("table", f"{bq_project}.{bq_dataset}.dim_tiempo").option("temporaryGcsBucket", "sutran-bucket-2025").mode("overwrite").save()

dim_tipo_via.write.format("bigquery").option("table", f"{bq_project}.{bq_dataset}.dim_tipo_via").option("temporaryGcsBucket", "sutran-bucket-2025").mode("overwrite").save()

f_siniestro.write.format("bigquery").option("table", f"{bq_project}.{bq_dataset}.f_siniestro").option("temporaryGcsBucket", "sutran-bucket-2025").mode("overwrite").save()